## 1. Create the JSONL dataset for fine-tuning

In [21]:
import pandas as pd
import numpy as np
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [5]:
path = "/Users/zhenyabudnyk/PycharmProjects/LyrAIX/Thesis/Data Labeling/lyrics_views2k.csv"
df = pd.read_csv(path)

print(len(df))

1091881


In [6]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,artist,title,tag,year,views,part,lyrics,explicitness,schemes,Valence,Arousal,Dominance,topic
0,0,0,0,JAY-Z,Can I Live,rap,1996,468624,Verse,While I'm watchin' every nigga watchin' me clo...,Explicit content,0-0-0-0-1-1-0-2-0-3-4-5-0-6-7-8-7-9-7-5-5-10-1...,Medium,Medium,Medium,general
1,1,1,1,JAY-Z,Can I Live,rap,1996,468624,Chorus,Ge-ge-geyeahhh\nCan I live?\nCan I live?,Normal,0-1-2,High,High,High,life and relationships
2,2,2,2,JAY-Z,Can I Live,rap,1996,468624,Verse,My mind is infested with sick thoughts that ci...,Normal,0-1-1-1-2-3-4-5-5-6-6-1-1-1-7-7-8,Medium,Medium,High,general
3,3,3,3,JAY-Z,Can I Live,rap,1996,468624,Chorus,Can I live?\nCan I live?\nCan I live?\nCan I l...,Normal,0-1-2-3,High,High,High,life and relationships
4,4,4,4,Fabolous,Forgive Me Father,rap,2003,4743,Hook,Forgive me father for I have sinned\nBut look ...,Explicit content,0-0-0-0-1-0-0-0,High,Medium,High,life and relationships
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091876,5282452,5282452,5282452,Genius English Translations,Rammstein - Zick Zack English Translation,rock,2022,6568,Chorus,"Snip-snip, snip-snip, cut that off\nTick-tock,...",Normal,0-1-1,Medium,Medium,Low,general
1091877,5282479,5282479,5282479,Interpol,Toni,rock,2022,3439,Verse,"Flame down Pacific highway\nStill in shape, my...",Normal,0-1-2-3,Low,Low,Low,life and relationships
1091878,5282480,5282480,5282480,Interpol,Toni,rock,2022,3439,Chorus,I'd like to see them win\nI like the inspirati...,Normal,0-1-2-3-4-2,High,Low,High,money and authority
1091879,5282481,5282481,5282481,Interpol,Toni,rock,2022,3439,Verse,The aim now is perfection always\nThe aim now ...,Explicit content,0-1-0-2,High,Medium,High,life and relationships


In [11]:
df = df.drop(labels=["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2"] ,axis="columns")


In [13]:
df

,artist,title,tag,year,views,part,lyrics,explicitness,schemes,Valence,Arousal,Dominance,topic
0,JAY-Z,Can I Live,rap,1996,468624,Verse,While I'm watchin' every nigga watchin' me clo...,Explicit content,0-0-0-0-1-1-0-2-0-3-4-5-0-6-7-8-7-9-7-5-5-10-1...,Medium,Medium,Medium,general
1,JAY-Z,Can I Live,rap,1996,468624,Chorus,Ge-ge-geyeahhh\nCan I live?\nCan I live?,Normal,0-1-2,High,High,High,life and relationships
2,JAY-Z,Can I Live,rap,1996,468624,Verse,My mind is infested with sick thoughts that ci...,Normal,0-1-1-1-2-3-4-5-5-6-6-1-1-1-7-7-8,Medium,Medium,High,general
3,JAY-Z,Can I Live,rap,1996,468624,Chorus,Can I live?\nCan I live?\nCan I live?\nCan I l...,Normal,0-1-2-3,High,High,High,life and relationships
4,Fabolous,Forgive Me Father,rap,2003,4743,Hook,Forgive me father for I have sinned\nBut look ...,Explicit content,0-0-0-0-1-0-0-0,High,Medium,High,life and relationships
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091876,Genius English Translations,Rammstein - Zick Zack English Translation,rock,2022,6568,Chorus,"Snip-snip, snip-snip, cut that off\nTick-tock,...",Normal,0-1-1,Medium,Medium,Low,general
1091877,Interpol,Toni,rock,2022,3439,Verse,"Flame down Pacific highway\nStill in shape, my...",Normal,0-1-2-3,Low,Low,Low,life and relationships
1091878,Interpol,Toni,rock,2022,3439,Chorus,I'd like to see them win\nI like the inspirati...,Normal,0-1-2-3-4-2,High,Low,High,money and authority
1091879,Interpol,Toni,rock,2022,3439,Verse,The aim now is perfection always\nThe aim now ...,Explicit content,0-1-0-2,High,Medium,High,life and relationships


In [14]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict
import random

ModuleNotFoundError: No module named 'tiktoken'

In [19]:
SYSTEM_PROMPT = """You are a Lyrics Writing Assistant, adept at creating song lyrics with specific guidelines. When given an artist's name, use it to capture the essence of their unique style, crafting lyrics that reflect their artistic identity. You will also receive characteristics of the song part (e.g., Chorus, Verse, Bridge, Hook), along with a rhyming scheme (e.g., 0-0-1-1, where the same numbers indicate rhyming lines).

In addition, be mindful of the explicitness parameter: if the provided text is marked as 'explicit', your generated lyrics should also contain explicit content; if it's marked as 'non-explicit', ensure the lyrics are without explicit content. This aligns the lyrics with the intended audience and content sensitivity.

You are also to incorporate emotional characteristics: valence (happiness level of the text, on a scale from low to high), arousal (activity level of the mood, from low to high), and dominance (assertiveness of the message, from low to high). Generalized topics for the lyrics will be provided, which should guide the theme or subject matter.

Your primary task is to generate new, authentic lyrics that align with these defined characteristics, focusing on creativity and adherence to the guidelines. The goal is to produce lyrics that resonate with the artist's style and the specified emotional and thematic parameters, ensuring relevance and originality in your lyrical composition."""
with open('fine_tuning.jsonl', 'w') as file:
    for i, row in df.iterrows():
        data = {
    "messages": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f'Generate a new {row["part"]} of a song lyrics in the style of {row["artist"]}. The valence level should be'
                                    f' {row["Valence"]}, arousal level should be {row["Arousal"]} and dominance level should be {row["Dominance"]}'
                                    f'. The generated lyrics should have {"explicit content" if row["explicitness"] else "non-explicit content"}. The approximate rhyming scheme should be {row["schemes"]}.'
                                    f'The topic of the lyrical text should be {row["topic"]}.'},
        {"role": "assistant", "content":  row["lyrics"]}
    ]
}
        json_str = json.dumps(data)
        file.write(json_str + '\n')

## 2. Check the fine-tuning data

In [18]:
data_path = "/Users/zhenyabudnyk/PycharmProjects/LyrAIX/application/fine_tuning.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 1091881
First example:
{'role': 'system', 'content': "You are a Lyrics Writing Assistant, adept at creating song lyrics with specific guidelines. When given an artist's name, use it to capture the essence of their unique style, crafting lyrics that reflect their artistic identity. You will also receive characteristics of the song part (e.g., Chorus, Verse, Bridge, Hook), along with a rhyming scheme (e.g., 0-0-1-1, where the same numbers indicate rhyming lines).\n\nIn addition, be mindful of the explicitness parameter: if the provided text is marked as 'Explicit', your generated lyrics should also contain explicit content; if it's marked as 'Normal', ensure the lyrics are without explicit content. This aligns the lyrics with the intended audience and content sensitivity.\n\nYou are also to incorporate emotional characteristics: valence (happiness level of the text, on a scale from low to high), arousal (activity level of the mood, from low to high), and dominance (assertiv

In [22]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [23]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")